# Drift Example

Simple 1 m drift based on the benchmark in:

In [ ]:
%config InlineBackend.figure_format='retina'

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from impact import ImpactZ, ImpactZInput
from impact.z.input import (
    Drift,
    DistributionZType,
    IntegratorType,
    DiagnosticType,
    OutputZType,
    BoundaryType,
    WriteFull,
)

reference_particle_mass = 510998.9461

# TODO write header showing repr as well
input = ImpactZInput(
    # line 1
    ncpu_y=1,
    ncpu_z=1,
    # line 2
    seed=6,
    n_particle=100000,
    integrator_type=IntegratorType.linear,  # TODO: support strings?
    err=1,
    diagnostic_type=DiagnosticType.at_given_time,
    output_z=OutputZType.extended,
    # line 3
    nx=32,
    ny=32,
    nz=32,
    boundary_type=BoundaryType.trans_open_longi_open,
    radius_x=0.15,  # particles die after this point
    radius_y=0.15,
    z_period_size=0.0,  # TODO we think this is unused based on fortran code reading
    # line 4
    distribution=DistributionZType.gauss,
    restart=0,
    subcycle=0,  # TODO what is this?
    nbunch=1,
    # line 5
    particle_list=[100000],  # note: this may be a way of having different species
    # line 6
    current_list=[0.0],
    # line 7
    charge_over_mass_list=[
        1.0 / reference_particle_mass
    ],  # protons=>1.06577993775845e-09  electrons=1/.511e6 (?) (charge_over_mass)
    # line 8
    twiss_alpha_x=0.0,
    twiss_beta_x=1.0,
    twiss_norm_emit_x=1e-6,
    twiss_mismatch_x=1.0,
    twiss_mismatch_px=1.0,
    twiss_offset_x=0.0,
    twiss_offset_px=0.0,
    # line 9
    twiss_alpha_y=0.0,
    twiss_beta_y=1.0,
    twiss_norm_emit_y=1e-6,
    twiss_mismatch_y=1.0,
    twiss_mismatch_py=1.0,
    twiss_offset_y=0.0,
    twiss_offset_py=0.0,
    # line 10
    twiss_alpha_z=0.0,
    twiss_beta_z=1.0,
    twiss_norm_emit_z=1e-6,
    twiss_mismatch_z=1.0,
    twiss_mismatch_e_z=1.0,
    twiss_offset_phase_z=0.0,
    twiss_offset_energy_z=0.0,
    # line 11
    average_current=1.0,
    initial_kinetic_energy=9489001.0539,  # eV -> 10MeV total energy
    reference_particle_mass=reference_particle_mass,
    reference_particle_charge=-1.0,
    scaling_frequency=1e9,  # arbitrarily set to 1GHz here
    initial_phase_ref=0.0,
    lattice=[
        WriteFull(file_id=2000),
        # Drift(length=1e-9, steps=10, map_steps=10),  # <- to get initial particles, set this and set average_current to 0.0
        Drift(length=1.0, steps=10, map_steps=10),
    ],
    # filename=PosixPath('/Users/klauer/Repos/lume-impact/docs/examples/z/example1/ImpactZ.in')
)

In [ ]:
results = {}
npart = 1000
input.n_particle = npart
input.particle_list = [npart]
I = ImpactZ(input, use_temp_dir=False, workdir="./tmp")
output = I.run(verbose=True)

results

In [ ]:
I.output

In [ ]:
keys = list(I.output)
fig, axs = plt.subplot_mosaic(list(zip(keys[::2], keys[1::2])), figsize=(12, 64))

for key in keys:
    try:
        I.output.plot(key, ax=axs[key])
    except Exception as ex:
        print("failed to plot key", key, type(ex), str(ex))

fig.tight_layout()